# NLP-HW3-Part1&2
## Sepehr Ghobadi, Vida Ramezanian

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os, itertools, tqdm, codecs, random, pickle
from collections import Counter
import numpy as np
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F
import torch.utils.data as D
DRIVE_PATH = "/content/drive/MyDrive/NLP/HW3"

In [4]:
from importlib.machinery import SourceFileLoader
ModelsModule = SourceFileLoader("ModelsModule", DRIVE_PATH+'/Models.py').load_module()
DatasetsModule = SourceFileLoader("DatasetsModule", DRIVE_PATH+'/Datasets.py').load_module()
from ModelsModule import LSTM, SiameseLSTM, myword2vec
from DatasetsModule import MasnaviDataset, RhymeBatchSampler

In [95]:
!pip3 install arabic_reshaper
!pip3 install python-bidi
import arabic_reshaper
from matplotlib import font_manager as fm, rcParams
from bidi.algorithm import get_display

def make_farsi_text(x):
    reshaped_text = arabic_reshaper.reshape(x)
    farsi_text = get_display(reshaped_text)
    return farsi_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Utils

In [5]:
PERSIAN_EMBEDDINGS = "اأآبپتثجچحخدذرزژسشصضطظعغفقکگلمنوهیئ"

def do_rhyme_words(w1,w2):
    return w1.endswith(w2) or w2.endswith(w1) or w2 == w1 or (w1[-2:]==w2[-2:] and w1[-2:]!='ست')

def do_rhyme_mesras(m1, m2):
    return do_rhyme_words( m1[-1], m2[-1] )

def is_masnavi(curr_beyt, next_beyt):
    if not do_rhyme_mesras(*curr_beyt):
        return False
    if (next_beyt is not None) and not do_rhyme_mesras(*next_beyt):
        return False
    return True

def get_beyt_rhyme(mesra1, mesra2):
    for idx in range(1,min(len(mesra1),len(mesra2))):
        if do_rhyme_words(mesra1[-idx], mesra2[-idx]) and mesra1[-idx] != mesra2[-idx]:
            return tuple(sorted((mesra1[-idx], mesra2[-idx])))
    return None

def vec2word(chars):
    idx = 0
    while(idx < len(chars)) and chars[idx]=="__PAD__":
        idx+=1
    chars = chars[idx:]
    spec_idxs = [0]
    for idx, c in enumerate(chars):
        if "_" in c:
            spec_idxs.append(idx)
    spec_idxs.append(-1)
    ans = []
    for idx in range(len(spec_idxs)-1):
      word = ''.join(chars[spec_idxs[idx]+1:spec_idxs[idx+1]])
      ans.append(chars[spec_idxs[idx]])
      if len(word)>0:
        ans.append(word)
    return ans

# Dataset

در ابتدا قالب های مختلف جدا شده بودند و قصد داشتیم که مدل های مختلف را روی قالب های مختلف آموزش دهیم اما به دلیل محدودیت ها محاسباتی قادر به انجام این کار نبودیم و مدل های فقط روی اشعار فردوسی اموزش داده شده اند.

**Skip Bellow Cells And Only Run Loader Cell**

In [68]:
mesras = {}
for filename in os.listdir(DRIVE_PATH+'/Persian_poems_corpus/normalized'):
    mesras[filename[:-9]] = list(filter(lambda m:len(m)>2, [x.strip().split() for x in codecs.open(DRIVE_PATH+f'/Persian_poems_corpus/normalized/{filename}','rU','utf-8').readlines()]))
with open(DRIVE_PATH+'/datasets/mesras.pickle', 'wb') as f:
    pickle.dump(mesras, f)

In [77]:
masnavis = []
for poet,p_mesras in mesras.items():
    if poet != "ferdousi":
        continue
    for idx in range(0,len(p_mesras),2):
        if idx+1 >= len(p_mesras):
            break
        curr_beyt = (p_mesras[idx], p_mesras[idx+1])
        next_beyt = None if idx+3 >= len(p_mesras) else (p_mesras[idx+2], p_mesras[idx+3])
        if is_masnavi(curr_beyt, next_beyt):
            masnavis.append(curr_beyt)
with open(DRIVE_PATH+'/datasets/masnavis.pickle', 'wb') as f:
    pickle.dump(masnavis, f)

In [70]:
qazals = []
for poet,p_mesras in mesras.items():
    idx = 0
    while idx < len(p_mesras):
        if idx+1 >= len(p_mesras):
            break
        curr_beyt = (p_mesras[idx], p_mesras[idx+1])
        idx += 2
        if do_rhyme_mesras(*curr_beyt):
            qazal = [curr_beyt]
            next_beyt = None if idx+1 >= len(p_mesras) else (p_mesras[idx], p_mesras[idx+1])
            while (next_beyt is not None) and do_rhyme_mesras(next_beyt[1], curr_beyt[1]):
                qazal.append(next_beyt)
                idx+=2
                next_beyt = None if idx+1 >= len(p_mesras) else (p_mesras[idx], p_mesras[idx+1])
            if len(qazal) > 1:
                qazals.append(qazal)
with open(DRIVE_PATH+'/datasets/qazals.pickle', 'wb') as f:
    pickle.dump(qazals, f)

In [71]:
rhymes = []
for beyt in masnavis:
    rhymes.append( get_beyt_rhyme(*beyt) )
for qazal in qazals:
    rhymes.append(get_beyt_rhyme(*qazal[0]))
   
    #option 1: get rhymes of each beyt with first beyt of qazal
    for idx in range(1,len(qazal)):
        rhymes.append( get_beyt_rhyme(qazal[0][1], qazal[idx][1]) )
    
    
    # #option2: get rhymes of each beyt with next beyt of qazal
    # for idx in range(0,len(qazal)-1):
    #     rhymes.append( get_beyt_rhyme(qazal[idx][1], qazal[idx+1][1]) )
    
    # option 3: get rhymes of each pair of beyts in qazal
#     for idx1 in range(0,len(qazal)):
#         for idx2 in range(idx1+1,len(qazal)):
#             rhymes.append( get_beyt_rhyme(qazal[idx1][1], qazal[idx2][1]) )
rhymes = list(set([r for r in rhymes if (r is not None)]))
with open(DRIVE_PATH+'/datasets/rhymes.pickle', 'wb') as f:
    pickle.dump(rhymes, f)

## Datasets Loader

In [78]:
with open(DRIVE_PATH+'/datasets/mesras.pickle', 'rb') as f:
    mesras = pickle.load(f)
with open(DRIVE_PATH+'/datasets/masnavis.pickle', 'rb') as f:
    masnavis = pickle.load(f)
with open(DRIVE_PATH+'/datasets/qazals.pickle', 'rb') as f:
    qazals = pickle.load(f)
with open(DRIVE_PATH+'/datasets/rhymes.pickle', 'rb') as f:
    rhymes = pickle.load(f)

### Chars Embedding

In [73]:
#!pip3 install gensim=="4.2.0"
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

class CharEmbedding(object):
    def __init__(self, corpus):
        super(CharEmbedding, self).__init__()
        self.corpus = corpus
        self.emb_dim = len(corpus)
        self.embeddings = {}
        for idx, w in enumerate(corpus):
            self.embeddings[w] = torch.FloatTensor([int(i == idx) for i in range(len(corpus))])

    def __call__(self, words):
        if isinstance(words, str):
            words = [words]
        embeddings = []
        for word in words:
            if word in self.embeddings:
                embeddings.append(self.embeddings[word])
            else:
                embeddings.append( torch.zeros(len(self.corpus)) )
        return torch.vstack(embeddings)

### Corpus and Word Embeddings Model

we use 140,000 words in "unigrams" as corpus (we can use downloaded word2vec which have only 70,000 words as corpus but unks increas in train)

In [74]:
all_mesras = list(itertools.chain(*mesras["ferdousi"].values()))
unigrams = list(itertools.chain(*all_mesras))
unigrams = list(set([w for w in unigrams]))
char_unigrams = list(set(list(itertools.chain(*[[c for c in w] for w in unigrams]))))

special_words = ["__PAD__", "__BOM__", "__EOM__", "__START__", "__SPACE__"]
for w in special_words:
    unigrams.append(w)
    char_unigrams.append(w)

with open(DRIVE_PATH+'/datasets/unigrams.pickle', 'wb') as f:
    pickle.dump(unigrams, f)
with open(DRIVE_PATH+'/datasets/char_unigrams.pickle', 'wb') as f:
    pickle.dump(char_unigrams, f)

AttributeError: ignored

In [ ]:
with open(DRIVE_PATH+'/datasets/unigrams.pickle', 'rb') as f:
    unigrams = pickle.load(f)
with open(DRIVE_PATH+'/datasets/char_unigrams.pickle', 'rb') as f:
    char_unigrams = pickle.load(f)

## Rhyme Embedding

برای بهبود عملکرد مدل های زبانی آموزش داده شده می توان یک مدل بدون نظارت آموزش داد تا به میزان قافیه و هم آوا بودن کلمات امتیاز دهد. به کمک این مدل می توان در خروجی مدل های زبانی که معمولا یک توزیع روی دیکشنری هستند، به جای سمپل گرفتنن محتمل ترین توکن، توکن ها را سمپل گرفت و با مدل قافیه چک کرد یا توکن با بیشترین احتمال که امتیاز هم قافیه بودن آن با ورودی از حدی بیشتر است را انتخاب کرد.

آموزش این مدل هم به صورت بدون نظارت و توسط یک شبکه

siamese

که در سطح کاراکتر مقایسه دو شبکه ال اس تی ام را برای جفت ورودی ها شبیه سازی می کند انجام شده که لاس آن کسینوس بردار های امبدینگ جفت کلمات ورودی ضربدر لیبل آنها است. در نهایت در زمان پیش بینی میزان شباهت ۲ کلمه ضرب داخلی بردار های امبدیینگ یاد گرفته شده ی انها است. پیاده سازی این مدل در فایل

Models.py/SiameseLSTM

در دسترس است.

### Preparing Rhymes Dataset

به ازای هر جفت قافیه و برای هرر کلمه آن تعدادی کلمه رندوم به عنووان داده بدون قافیه به دیتاست اضافه می کنیم

In [36]:
NON_RHYME_DATA_POINT = 1

rhymes_dataset = [(w1,w2,1) for (w1,w2) in rhymes]
for w1,w2 in rhymes:
    for idx in range(NON_RHYME_DATA_POINT):
        x = None
        while True:
            x = random.sample(unigrams,1)[0]
            if not do_rhyme_words(w1,x):
                break
        rhymes_dataset.append( (w1,x,-1) )
        while True:
            x = random.sample(unigrams,1)[0]
            if not do_rhyme_words(w2,x):
                break
        rhymes_dataset.append( (x,w2,-1) )

In [37]:
random.sample(rhymes_dataset,10)

[('سزاوارشان', 'بریدش', -1),
 ('سخنگوی', 'چهار', -1),
 ('داد', 'یاد', 1),
 ('جان', 'نگهبان', 1),
 ('دیرساز', 'گداز', 1),
 ('چویابد', 'ریختند', -1),
 ('همچنین', 'پرسیدنی', -1),
 ('خروش', 'مکوش', 1),
 ('لواده', 'کرانه', -1),
 ('درکنی', 'کشت', -1)]

In [38]:
seed = 42
train_length, test_length = (int)(0.95*len(rhymes_dataset)), len(rhymes_dataset)-(int)(0.95*len(rhymes_dataset))
rhymes_dataset_train, rhymes_dataset_test = torch.utils.data.random_split(rhymes_dataset, [train_length, test_length],torch.Generator().manual_seed(seed))


### Rhymes Model

In [39]:
def run_rhyme_model(rhyme_model, data_loader, batch_size, device, train=True):
    print(f"Running on {device}")
    accs, losses = [], []
    for idx, batch in enumerate(data_loader):
        rhyme_model.optimizer.zero_grad()
        
        r1 = torch.vstack([myword2vec(batch[0][b]) for b in range(batch_size)]).to(device)
        r2 = torch.vstack([myword2vec(batch[1][b]) for b in range(batch_size)]).to(device)
        y = torch.Tensor([batch[2][b] for b in range(batch_size)]).to(device)
        embedding1, embedding2 = rhyme_model(r1,r2,y)
        loss = rhyme_model.get_loss(embedding1.view(batch_size,-1), embedding2.view(batch_size,-1),y)
        losses.append(loss)

        y_pred = [1 if l<0.3 else -1 for l in loss]
        acc = sum([int(y[idx]==y_pred[idx]) for idx in range(len(y))])/len(y)
        accs.append(acc)

        if train:
            loss = loss.mean()
            loss.backward()
        
            rhyme_model.optimizer.step()
        
            if idx%1000 == 0:
                print(f"Train Loss: {loss}.         Train Acc: {acc}")
                torch.save(rhyme_model.state_dict(), DRIVE_PATH+f'/rhyme_models/rhyme_model{idx}.pt')
    return accs, losses

In [40]:
#!rm -rf $DRIVE_PATH/rhyme_models/*

#### Train

In [41]:
rhm_batch_size=256
rhm_train_iterations = 20000 # approx 5 epoch
rhm_device = 'cpu'

rhyme_model = SiameseLSTM(embedding_dim=128, hidden_dim=512, num_layers=1, dropout=0.0, learning_rate=0.01, device=rhm_device)

rhymes_train_sampler = RhymeBatchSampler(rhymes_dataset_train, npratio=0.5, iterations=rhm_train_iterations, batch_size=rhm_batch_size) #npratio=1/(1+NON_RHYME_DATA_POINT)
rhymes_train_data_loader = torch.utils.data.DataLoader(rhymes_dataset_train, batch_sampler=rhymes_train_sampler)

#train_accs, train_losses = run_rhyme_model(rhyme_model, rhymes_train_data_loader, rhm_batch_size, rhm_device, train=True)


#### Eval

In [42]:
rhyme_model = SiameseLSTM(embedding_dim=128, hidden_dim=512, num_layers=1, dropout=0.0, learning_rate=0.01, device=rhm_device)
rhymes_test_sampler = RhymeBatchSampler(rhymes_dataset_test, npratio=0.5, iterations=2000, batch_size=rhm_batch_size)
rhymes_test_data_loader = torch.utils.data.DataLoader(rhymes_dataset_test, batch_sampler=rhymes_test_sampler)

rhyme_model.load_state_dict(torch.load(DRIVE_PATH+'/rhyme_models/rhyme_model19000.pt', map_location=torch.device(rhm_device)))

# rhyme_model.eval()
# eval_accs, eval_loss = run_rhyme_model(rhyme_model, rhymes_test_data_loader, rhm_batch_size, rhm_device, train=False)
# print(f"Eval Accuracy: {mean(eval_accs)}")

<All keys matched successfully>

دقت مدل حدود ۸۰ درصد بود (البته روی دیتاست ساختگی که دقیق نیست و به صورت قانون-محور و بدون برچسب زنی از دیتا ست اشعار استخراج شده است)

In [63]:
for batch in rhymes_test_data_loader:
    r1 = [batch[0][b] for b in range(rhm_batch_size)]
    r2 = [batch[1][b] for b in range(rhm_batch_size)]
    y = [batch[2][b] for b in range(rhm_batch_size)] 
    for idx in range(10):
        print(f"r1: {r1[idx]}    r2:{r2[idx]}.     label:{y[idx]}.     model score:{rhyme_model.predict(r1[idx],r2[idx])}")
    break

r1: ارغوان    r2:خامی.     label:-1.     model score:tensor([0.])
r1: براورنگ    r2:پرداختن.     label:-1.     model score:tensor([3.3067e-13])
r1: جای    r2:خدای.     label:1.     model score:tensor([1.0000])
r1: بازارگان    r2:بیچارگان.     label:1.     model score:tensor([1.0000])
r1: بگریختند    r2:نکشت.     label:-1.     model score:tensor([0.0136])
r1: آذرپرست    r2:نامدار.     label:-1.     model score:tensor([0.0146])
r1: آبروی    r2:خوی.     label:1.     model score:tensor([0.9939])
r1: قند    r2:چونستود.     label:-1.     model score:tensor([0.6795])
r1: برگزید    r2:سزید.     label:1.     model score:tensor([0.9994])
r1: وار    r2:گذار.     label:1.     model score:tensor([0.9701])


# N-Gram Language Model

In [100]:
from itertools import product
import math
import nltk


class LanguageModel(object):
    SOS = "__BOM__"
    EOS = "__EOM__"
    UNK = "<UNK>"
    
    def __init__(self, train_data, n, laplace=1):
        self.n = n
        self.vocab = dict()
        self.laplace = laplace
        self.tokens = self.preprocess(train_data, n)
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

    def _smooth(self):
        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }

    def _create_model(self):
        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def perplexity(self, test_data):
        test_tokens = self.preprocess(test_data, self.n)
        test_ngrams = nltk.ngrams(test_tokens, self.n)
        N = len(test_tokens)

        known_ngrams  = [self._convert_oov(ngram) for ngram in test_ngrams]
        probabilities = [self.model[ngram] for ngram in known_ngrams]
        
        # for x,y in zip(known_ngrams, probabilities):
        #     print(x,y)
        
        return math.exp((-1/N) * sum(map(math.log, probabilities)))

    def _best_candidate(self, prev, without=[]):
        
        blacklist  = [LanguageModel.UNK] + without

        if len(prev) < self.n:
            prev = [LanguageModel.SOS]*(self.n-1)

        candidates = list(((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==tuple(prev)))

        probs = [y for x,y in candidates]
        probs = probs/np.sum(probs)
        words = [x for x,y in candidates]

        idx = np.random.choice(len(words), 1, replace=False, p=probs)[0]
        
        while words[idx] in blacklist:
            idx = np.random.choice(len(words), 1, replace=False, p=probs)[0]
        
        return (words[idx], probs[idx])
         
    def generate_sentence(self, input, min_len=12, max_len=24):
        #sent, prob = ([LanguageModel.SOS] * (max(1, self.n-1)), 1)
        sent, prob, start = input.copy(), 1, True
        while sent[-1] != LanguageModel.EOS or start:
            start = False
            prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
            blacklist = sent + ([LanguageModel.EOS,LanguageModel.SOS] if len(sent) < min_len else [])
            next_token, next_prob = self._best_candidate(prev, without=blacklist)
            sent.append(next_token)
            prob *= next_prob

            if len(sent) >= max_len:
                sent.append(LanguageModel.EOS)

        #return (' '.join(sent[(self.n-1):-1]), -1/math.log(prob))
        #return (' '.join(sent), -1/math.log(prob))
        return ' '.join(sent)
    
    

    def add_sentence_tokens(self, sentences, n):
        sos = ' '.join([LanguageModel.SOS] * (n-1)) if n > 1 else LanguageModel.SOS
        return ['{} {} {}'.format(sos, s, LanguageModel.EOS) for s in sentences]

    def replace_singletons(self, tokens):
        if len(self.vocab) == 0:
            self.vocab = nltk.FreqDist(tokens)
        return [token if self.vocab[token] > 1 else LanguageModel.UNK for token in tokens]

    def preprocess(self, sentences, n):
        sentences = self.add_sentence_tokens(sentences, n)
        tokens = ' '.join(sentences).split()
        tokens = self.replace_singletons(tokens)
        return tokens    

In [97]:
b = list(itertools.chain(*masnavis))
masnavi_beyts_str = [' '.join(b[2*idx])+' __EOM__ __BOM__ '+' '.join(b[2*idx+1]) for idx in range(len(b)//2)]
masnavi_beyts_str[0]

'به نام خداوند جان و خرد __EOM__ __BOM__ کزین برتر اندیشه برنگذرد'

In [79]:
masnavi_betys_train = masnavi_beyts_str[:int(0.9*len(masnavi_beyts_str))]
masnavi_betys_test = masnavi_beyts_str[int(0.9*len(masnavi_beyts_str)):]

حال عملکرد مدل به ازای طول های مختلف ترکیبات را بررسی میکنیم:

In [104]:
for n in [1,2,3,4,5]:
    print(f"\n ======== N-Gram Model:  n={n} ======== \n")
    n_gram_model = LanguageModel(masnavi_betys_train, n)
    sample_beyts = np.random.choice(masnavi_betys_test,3)
    for sbeyt in sample_beyts:
        first_mesra_tokens = sbeyt.split(" ")[0:sbeyt.split(" ").index("__EOM__")]
        print("input: "," ".join(first_mesra_tokens))
        print("model output:")
        print(n_gram_model.generate_sentence(first_mesra_tokens, 2*len(first_mesra_tokens)-2, 2*len(first_mesra_tokens)+2))
    print(f"\n perplexity of {n}-gram: {n_gram_model.perplexity(masnavi_betys_test)} \n")



 ======== N-Gram Model:  n=1 ======== 

input:  علم معنی را زمعنیها بپرس
model output:
علم معنی را زمعنیها بپرس است درمانده زبان گشته از __BOM__ آن __EOM__
input:  مرد باید خواه خاص و خواه عام
model output:
مرد باید خواه خاص و خواه عام از اندازم چو دریغ قارن پیمودن پس آورد __BOM__ __EOM__
input:  روز و شب در خدمتش بودند شاد
model output:
روز و شب در خدمتش بودند شاد من معنی چارپای بسان گشاید روح که __EOM__

 perplexity of 1-gram: 460.6800640577724 


 ======== N-Gram Model:  n=2 ======== 

input:  او بسگ داد آن همه تا سگ بخورد
model output:
او بسگ داد آن همه تا سگ بخورد به پرستنده ببسته موج همی ور ماه کن اهل شده __EOM__
input:  چون خریدن را سزایی خون بود
model output:
چون خریدن را سزایی خون بود عقل بدو دلم پایی شرابی خانه ز سلک __EOM__
input:  جوهری یابی ز استغنای حق
model output:
جوهری یابی ز استغنای حق نه کارنامه فرود ای یعنی خرد تو __EOM__

 perplexity of 2-gram: 503.59973645029254 


 ======== N-Gram Model:  n=3 ======== 

input:  روی زردم زین غم و جامه کبود
model output:
روی زردم 

همانگونه که میبینیم با افزایش طول کانتکست، کیفییت ممکن است افزایش یابد اما نیاز به دیتا ست مناسب برای جلوگیری از اورفیتینگ وجود دارد. در کل کیفییت ایین مدل نمی تواند در تشخیص کانتکتس و تولید مصراع مناسب با مدل هایی مبتنی برشبکه های عصبی رقابت کند

# Encoder - Decoder

پیاده سازی مدل انکدر-دیکدر به روش های مختلفی انجام شد. از تکنیک های مختلف مکانیزم اتنشن مانند 

bahdanau

تا معماری های ال اس تی ام ماند چند لایه بودن یا دو طرفه بودن. همچنین برای تبدیل های ورودی و خروجی از تبدیل های خطی و همچنین شبکه های عصبی چند لایه استفاده شد.
مقادیر مختلف هایپر پارامتر بررای نرخ یادگیری و حد برش گرادیان ها استفاده شد اما متاسفانه باگ موجود در مدل که احتمالا در قسمت حاسبه لاس اتفاق می افتد را پیدا نکردیم.

In [13]:
class Encoder(nn.Module):
    def __init__(self, embed_dims, hidden_dims, num_layers, dropout, device):
        super(Encoder, self).__init__()
        self.device = device
        self.enc = LSTM(embed_dims, hidden_dims, num_layers, dropout, device, bidirectional=True)

    def forward(self, input):
        N, L = input.size(0), input.size(1)
        hidden, cell = self.enc.init_hidden(N)
        output, hidden, cell = self.enc(input, N, hidden, cell)
        return output, hidden, cell
    
class Decoder(nn.Module):
    def __init__(self, corpus, emb_dims, hidden_dims, output_dims, num_layers, dropout, device):
        super(Decoder, self).__init__()
        self.corpus = corpus
        self.device = device
        self.emb_dims = emb_dims
        self.dec = LSTM(emb_dims, 2*hidden_dims, num_layers, dropout=0.0, device=device, bidirectional=False)
        self.output_net = nn.Linear(4*hidden_dims,output_dims).to(device)
        # self.output_net = nn.Sequential( 
        #     nn.Linear(4*hidden_dims,4*hidden_dims),
        #     nn.ReLU(),
        #     nn.Linear(4*hidden_dims,output_dims),
        #   ).to(device)

        self.q_att = nn.Linear(4*hidden_dims,2*hidden_dims).to(device)
        # self.q_att = nn.Sequential( 
        #     nn.Linear(4*hidden_dims,4*hidden_dims),
        #     nn.ReLU(),
        #     nn.Linear(4*hidden_dims,2*hidden_dims),
        #   ).to(device)
        self.v_att = torch.nn.Parameter(torch.FloatTensor(2*hidden_dims)).to(device)
        torch.nn.init.normal_(self.v_att)

    def forward(self, dec_input, enc_outputs, hidden, cell):
        N = dec_input.size(0)
        dec_output, dec_hidden, dec_cell = self.dec(dec_input, N, hidden, cell)

        e_att = torch.cat((enc_outputs, dec_output.expand_as(enc_outputs)), dim=2)
        att_weights = torch.sum(self.v_att * torch.tanh(self.q_att(e_att)), dim=2)
        att_context = torch.bmm(att_weights.unsqueeze(1), enc_outputs).squeeze(1)
        dec_output = torch.cat([att_context, dec_output.squeeze(dim=1)], dim=-1)

        softmax = nn.Softmax(dim=-1)
        dec_pred = softmax(self.output_net(dec_output))
        return dec_pred, dec_hidden, dec_cell
        
class EncoderDecoder(nn.Module):
    def __init__(self, corpus, input_dims, emb_dims, hidden_dims, output_dims, num_layers, dropout, device):
        super(EncoderDecoder, self).__init__()
        self.device = device
        self.corpus = corpus
        self.input_dims = input_dims
        self.emb_dims = emb_dims
        self.embedding = nn.Sequential( nn.Linear(self.input_dims, self.emb_dims) ).to(device)
        self.encoder = Encoder(self.emb_dims, hidden_dims, num_layers, dropout, device)
        self.decoder = Decoder(corpus, self.emb_dims, hidden_dims, output_dims, num_layers, dropout, device)
        self.loss_fn = nn.CrossEntropyLoss(reduction="none")
        
    def forward(self, input, target, target_mask, teacher_force_prob=1.0):
        input = self.embedding(input)
        enc_outputs, enc_hidden, enc_cell = self.encoder(input)
        enc_hidden = torch.hstack([*torch.split(enc_hidden, 2, dim=0)[0]]).unsqueeze(0)
        enc_cell = torch.hstack([*torch.split(enc_cell, 2, dim=0)[0]]).unsqueeze(0)
        
            
        N, L = target.size(0), target.size(1)
        start_token = "__BOM__"
        dec_input = torch.stack([torch.Tensor([self.corpus.index(start_token)]*N)]).transpose(0,1).unsqueeze(dim=-1).to(self.device)
        dec_hidden, dec_cell = enc_hidden, enc_cell
        pred = []
        loss = 0
        for l in range(1,L):
            dec_input = self.embedding(dec_input)
            dec_output, dec_hidden, dec_cell = self.decoder(dec_input, enc_outputs, dec_hidden, dec_cell)
            next_char = dec_output.argmax(dim=-1).float() #.view(-1,1,self.emb_dims)
            pred.append(next_char)
            step_loss = self.loss_fn(dec_output, target[:,l,:].squeeze(dim=1).type(torch.LongTensor).to(self.device))
            step_loss = [ls*int(l<target_mask[idx]) for idx, ls in enumerate(step_loss)]
            loss += sum(step_loss)
            tf = False if (teacher_force_prob < 1.0 and random.random() < 1.0-teacher_force_prob) else True
            dec_input = target[:,l,:].unsqueeze(dim=1) if tf else next_char.view(-1,1,1)

        return loss, torch.stack(pred)

In [28]:
from torch.profiler import profile, record_function, ProfilerActivity
from tqdm import tqdm

def train_encdec_model(encdec, dataLoaderTrain, datasetVal, encdec_optim, epochs, device):
    encdec.train()
    for epoch in range(epochs):
        print(f"=============epoch:{epoch}=============")
        for batch_idx, (mesras, (targets, targets_indices, masks)) in enumerate(dataLoaderTrain):
            encdec_optim.zero_grad()

            char_embeddings = CharEmbedding(char_unigrams)
            mesras = [m.split("#") for m in mesras]
            mesras = torch.stack([torch.Tensor([char_unigrams.index(c) for c in w]) for w in mesras]).unsqueeze(-1).to(device)
            targets = [t.split("#") for t in targets]
            targets = torch.stack([torch.Tensor([char_unigrams.index(c) for c in w]) for w in targets]).unsqueeze(-1).to(device)
            targets_indices = targets_indices.to(device)

            loss, preds = encdec(mesras, targets, masks, teacher_force_prob=1.0)
            loss.backward()
            grads_zero = False
            for n, p in encdec.named_parameters():
              if not p.grad.any():
                grads_zero = True
            if grads_zero:
              print("grads zero?:", grads_zero)

            nn.utils.clip_grad_norm_(encdec.parameters(), 0.1)
            #nn.utils.clip_grad_norm_(encdec.parameters(), -0.4)
            encdec_optim.step()
            if batch_idx%100 ==0:
                print(f"batch {batch_idx} loss: {loss/preds.size(0)}")
                idx_val = np.random.randint(0, len(datasetVal))
                with torch.no_grad():
                  eval_mesra, eval_target = datasetVal[idx_val][0].split("#"), datasetVal[idx_val][1][0].split("#")
                  eval_mesra = torch.stack([torch.Tensor([char_unigrams.index(c) for c in eval_mesra])]).unsqueeze(-1).to(device)
                  eval_target = torch.stack([torch.Tensor([char_unigrams.index(c) for c in eval_target])]).unsqueeze(-1).to(device)
                  mask = [datasetVal[idx_val][1][2]]

                  _, preds = encdec(eval_mesra, eval_target, mask, teacher_force_prob=0.0)
                  print("input:", vec2word(datasetVal[idx_val][0].split("#")))
                  print("predict:", vec2word([char_unigrams[int(c.item())] for c in preds]))
                torch.save(encdec.state_dict(), DRIVE_PATH+f'/encdec_model_ferdousi.pt')
            #print(prof.key_averages(group_by_stack_n=10).table(sort_by="self_cpu_memory_usage", row_limit=10))

    return encdec

In [29]:
encdec_batch_size = 128
encdec_epochs = 150
encdec_lr = 0.01
encdec_device = 'cuda:0'

masnaviDatasetTrain = MasnaviDataset(masnavis[:int(0.9*len(masnavis))], char_unigrams)
masnaviDatasetVal = MasnaviDataset(masnavis[int(0.9*len(masnavis)):], char_unigrams)

masnaviDataLoaderTrain = D.DataLoader(masnaviDatasetTrain, encdec_batch_size, shuffle=True)
masnaviDataLoaderVal = D.DataLoader(masnaviDatasetVal, encdec_batch_size, shuffle=True)

encdec = EncoderDecoder(corpus=char_unigrams, input_dims=1, emb_dims=64, hidden_dims=1024, output_dims=len(char_unigrams), num_layers=1, dropout=0.0, device=encdec_device)
encdec_optim = torch.optim.Adadelta(encdec.parameters(), lr=encdec_lr, rho=0.95, eps=1e-6)
#encdec_optim = torch.optim.SGD(encdec.parameters(), lr=encdec_lr)
encdec = train_encdec_model(encdec, masnaviDataLoaderTrain, masnaviDatasetVal, encdec_optim, encdec_epochs, encdec_device)

=============epoch:0=============
batch 0 loss: 200.58827209472656
input: ['__BOM__', '__BOM__', 'همه', '__SPACE__', 'مهتران', '__SPACE__', 'پشت', '__SPACE__', 'برگاشتند', '__EOM__']
predict: ['ا', 'اااااااااااااااااااااااجاجاجاجانننننننننننننههههههههههههه']
batch 100 loss: 200.1668701171875
input: ['__BOM__', '__BOM__', 'سپاهی', '__SPACE__', 'بران', '__SPACE__', 'گونه', '__SPACE__', 'کردی', '__SPACE__', 'تباه', '__EOM__']
predict: ['ا', 'اااااااااااااااااااااا', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', 'ن', '__EOM__', 'ن', '__EOM__', 'هننن', '__EOM__', 'نننننننههههههههههههه']
batch 200 loss: 199.24209594726562
input: ['__BOM__', '__BOM__', 'یکایک', '__SPACE__', 'همه', '__SPACE__', 'کار', '__SPACE__', 'او', '__SPACE__', 'را', '__SPACE__', 'بساخت', '__EOM__']
predict: ['ا', 'اااااااااااااااااااا', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '__EOM__', '_

KeyboardInterrupt: ignored